# Bespokefit workshop live session

## Learning objectives

By the end of this workshop you will be able to:

- 1. Build, configure and save a general bespokefit optimisation workflow
- 2. Build molecule specific optimizaion schemas from the general workflow
- 3. Load QCArchive computed data using OpenFF-QCSubmit as a fitting reference
- 4. Optimize some bespoke torsion parameters
- 5. analysis the results from ForceBalance 
- 6. Generate refernce data locally using xtb on the fly

In [1]:
from openff.qcsubmit.results import TorsionDriveResultCollection
from openff.bespokefit.workflows import BespokeWorkflowFactory
from openff.bespokefit.schema.optimizers import ForceBalanceSchema
from openff.bespokefit.schema.targets import TorsionProfileTargetSchema
from openff.bespokefit.bespoke import Executor
from openff.bespokefit.schema.data import BespokeQCData
from openff.qcsubmit.common_structures import QCSpec
from openff.toolkit.topology import Molecule
from qcportal import FractalClient
from pprint import pprint

## 1. Building the general workflow

Bespokefit aims to provide a reproducable parameter optimization workflow for SMIRNOFF based force fields. As such normal bespokefit execution starts with a general fitting workflow. This captures every process in the workflow along with any ajustable settings such as how the reference data should be generated. Lets start with a basic workflow which should be ready to use for torsion fitting.

In [2]:
# start with an empty workflow 
workflow = BespokeWorkflowFactory(fragmentation_engine=None, parameter_settings=[], target_smirks=[], target_templates=[])
pprint(workflow.dict())

{'expand_torsion_terms': True,
 'fragmentation_engine': None,
 'generate_bespoke_terms': True,
 'initial_force_field': 'openff_unconstrained-1.3.0.offxml',
 'optimizer': {'adaptive_damping': 1.0,
               'adaptive_factor': 0.2,
               'eigenvalue_lower_bound': 0.01,
               'error_tolerance': 1.0,
               'extras': {},
               'finite_difference_h': 0.01,
               'gradient_convergence_threshold': 0.01,
               'initial_trust_radius': -0.25,
               'job_type': 'optimize',
               'max_iterations': 10,
               'minimum_trust_radius': 0.05,
               'n_criteria': 2,
               'normalize_weights': False,
               'objective_convergence_threshold': 0.01,
               'penalty_additive': 1.0,
               'penalty_type': 'L1',
               'step_convergence_threshold': 0.01,
               'type': 'ForceBalance'},
 'parameter_settings': [],
 'target_smirks': [],
 'target_templates': []}


## 2. Build molecule specific schema



In [3]:
# load the molecules
mol = Molecule.from_file("data/input.sdf")

In [4]:
#mol[5]
mol

NGLWidget()

In [5]:
# load the default factory 
workflow = BespokeWorkflowFactory()

In [6]:
# process all molecules
#schema = workflow.optimization_schemas_from_molecules(mol[5])
schema = workflow.optimization_schemas_from_molecules(mol)

Building Fitting Schema :   0%|                           | 0/1 [00:00<?, ?it/s]Warning: Unable to load toolkit 'OpenEye Toolkit'. The Open Force Field Toolkit does not require the OpenEye Toolkits, and can use RDKit/AmberTools instead. However, if you have a valid license for the OpenEye Toolkits, consider installing them for faster performance and additional file format support: https://docs.eyesopen.com/toolkits/python/quickstart-python/linuxosx.html OpenEye offers free Toolkit licenses for academics: https://www.eyesopen.com/academic-licensing
[14:40:00] WARNING: Proton(s) added/removed
[14:40:00] WARNING: Proton(s) added/removed
[14:40:00] WARNING: Proton(s) added/removed
[14:40:00] WARNING: Proton(s) added/removed
[14:40:00] WARNING: Proton(s) added/removed
[14:40:00] WARNING: Proton(s) added/removed
Building Fitting Schema : 100%|███████████████████| 1/1 [00:30<00:00, 30.49s/it]
/Users/jeffreywagner/miniconda3/envs/2021-bespokefit-workshop/lib/python3.8/site-packages/openff/tool

In [7]:
# pull out the new smirks and show how they transfer between the parent and the fragment

In [8]:
# look into the data in the fitting schema
schema[0].targets[0].reference_data.dict(exclude={"tasks"})

{'type': 'bespoke',
 'qc_spec': {'method': 'B3LYP-D3BJ',
  'basis': 'DZVP',
  'program': 'psi4',
  'spec_name': 'default',
  'spec_description': 'Standard OpenFF optimization quantum chemistry specification.',
  'store_wavefunction': 'none',
  'implicit_solvent': None,
  'keywords': None},
 'target_conformers': 4}

## 3. Loading data from QCArchive

In [9]:
client = FractalClient()

In [10]:
client.list_collections("torsiondrivedataset")

tagline
collection          name                                                                                                 
TorsionDriveDataset Fragment Stability Benchmark                                                                     None
                    Fragmenter paper                                                                                 None
                    OpenFF Amide Torsion Set v1.0                       Amides, thioamides and amidines diversely func...
                    OpenFF Aniline 2D Impropers v1.0                    Substituted aniline derivatives with various e...
                    OpenFF DANCE 1 eMolecules t142 v1.0                                                              None
                    OpenFF Fragmenter Validation 1.0                                                                 None
                    OpenFF Full TorsionDrive Benchmark 1                                                             None
                    OpenFF Gen 2 Torsion Set 1 Roche                                                                 None
                    OpenFF Gen 2 Torsion Set 1 Roche 2                                                               None
                    OpenFF Gen 2 Torsion Set 2 Coverage                                                              None
                    OpenFF Gen 2 Torsion Set 2 Coverage 2                                                            None
                    OpenFF Gen 2 Torsion Set 3 Pfizer Discrepancy                                                    None
                    OpenFF Gen 2 Torsion Set 3 Pfizer Discrepancy 2                                                  None
                    OpenFF Gen 2 Torsion Set 4 eMolecules Discrepancy                                                None
                    OpenFF Gen 2 Torsion Set 4 eMolecules Discrepan...                                               None
                    OpenFF Gen 2 Torsion Set 5 Bayer                                                                 None
                    OpenFF Gen 2 Torsion Set 5 Bayer 2                                                               None
                    OpenFF Gen 2 Torsion Set 6 Supplemental                                                          None
                    OpenFF Gen 2 Torsion Set 6 Supplemental 2                                                        None
                    OpenFF Gen3 Torsion Set v1.0                                             OpenFF Gen3 Torsion Set v1.0
                    OpenFF Group1 Torsions                                                                           None
                    OpenFF Group1 Torsions 2                                                                         None
                    OpenFF Group1 Torsions 3                                                                         None
                    OpenFF Primary Benchmark 1 Torsion Set                                                           None
                    OpenFF Primary Benchmark 2 Torsion Set                                                           None
                    OpenFF Primary TorsionDrive Benchmark 1                                                          None
                    OpenFF Protein Fragments TorsionDrives v1.0         Torsiondrives for selected dihedrals of variou...
                    OpenFF Rowley Biaryl v1.0                                          Torsiondrives of biaryl molecules.
                    OpenFF Substituted Phenyl Set 1                                                                  None
                    OpenFF Substituted Phenyl Set 1 v2.0                Torsiondrives for selected dihedrals of variou...
                    OpenFF Theory Benchmarking Set B3LYP-D3BJ 6-31+Gss  Torsiondrives for benchmarking B3LYP-D3BJ/6-31...
                    OpenFF Theory Benchmarking Set B3LYP-D3BJ DZVP ...     Torsiondrives

In [11]:
# create a result from the dataset we know our molecule is in
result = TorsionDriveResultCollection.from_server(client, "OpenFF-benchmark-ligand-fragments-v1.0", "default")

In [12]:
# check how many results we have
result.n_molecules

368

In [13]:
result.n_results

481

So we have 368 unique molecules and 481 torsiondrives, with some molecules have multipule torsion driven. All scans are 1D however.

In [14]:
records_and_molecules = result.to_records()

In [15]:
# show what we have pulled down
record, torsion_molecule = records_and_molecules[1]

In [16]:
pprint(record.dict())

{'created_on': datetime.datetime(2020, 8, 11, 8, 41, 29, 323584),
 'error': None,
 'extras': {},
 'final_energy_dict': {'[-105]': -632.1082407250015,
                       '[-120]': -632.1100436475298,
                       '[-135]': -632.1116905968972,
                       '[-150]': -632.1124503467556,
                       '[-15]': -632.1122764941144,
                       '[-165]': -632.1122360012745,
                       '[-30]': -632.1124124915791,
                       '[-45]': -632.1114883980323,
                       '[-60]': -632.1097698299328,
                       '[-75]': -632.1080652765091,
                       '[-90]': -632.1073734464316,
                       '[0]': -632.1118147038835,
                       '[105]': -632.1082410704761,
                       '[120]': -632.1100441611275,
                       '[135]': -632.111691344029,
                       '[150]': -632.112451407927,
                       '[15]': -632.112275166978,
                    

In [17]:
records_and_molecules[4][1].to_file('data/input.sdf', file_format='sdf')

In [18]:
#torsion_molecule.to_file('data/input.sdf', file_format='sdf')
torsion_molecule

NGLWidget(max_frame=23)

In [19]:
for molecule in schema:
    molecule.update_with_results(records_and_molecules)

In [20]:
for molecule in schema:
    print(molecule.ready_for_fitting)

False


## 4. Optimize a bespoke torsion parameter

In [21]:
executor = Executor()

In [ ]:
executor.execute(schema[0])

Searching for reference tasks
Reference tasks found generating input datasets
generating task map
connecting to qcfractal


/Users/jeffreywagner/miniconda3/envs/2021-bespokefit-workshop/lib/python3.8/site-packages/openff/qcsubmit/validators.py:255: UserWarning: The molecule BrC6H4O3S has a net charge of -1.0.
  warnings.warn(
INFO:tornado.application:FractalServer:
INFO:tornado.application:    Name:          QCFractal Snowflake Instance
INFO:tornado.application:    Version:       v0.15.6
INFO:tornado.application:    Address:       http://localhost:50230/
INFO:tornado.application:    Database URI:  postgresql://localhost:50222/
INFO:tornado.application:    Database Name: temporary_snowflake
INFO:tornado.application:    Query Limit:   1000000

INFO:tornado.application:Waiting on queue_manager to build.
INFO:tornado.application:FractalServer successfully started.



FractalServer(name='QCFractal Snowflake Instance' uri='http://localhost:50230/')
